In [13]:
import numpy as np
import pandas as pd
from differences import ATTgt
from matplotlib import pyplot as plt
INPUT_DATA = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/data/03_cleaned/restricted.csv"
df = pd.read_csv(INPUT_DATA)

/var/folders/lm/j45l41m16cs8sd5r9sh4gj6w0000gn/T/ipykernel_77102/3882343986.py:6: DtypeWarning: Columns (149,151) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_DATA)


In [14]:
# Store column names containing Zestimates.
years = [str(year) for year in range(2013, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = ["2012-12"] + [str(year) + "-" + str(month) for year in years for month in months]
month_to_int_dictionary = {key: value for value, key in enumerate(value_vars)}
int_to_month_dictionary = {key: value for key, value in enumerate(value_vars)}

In [15]:
# Drop rows for which we have no Zestimates.
no_zestimates_mask = df[value_vars].notna().all(axis=1)  # True if we see every month after the end of 2017.
print(f"Limiting sample to {no_zestimates_mask.sum()} evictions for which we have Zestimate data.")
df = df.loc[no_zestimates_mask, :]

# Drop rows for which we do not have the latest docket date.
no_latest_docket_date_mask = ~df['latest_docket_date'].isna()
print(f"Limiting sample to {no_latest_docket_date_mask.sum()} evictions for which the latest docket date is listed.")
df = df.loc[no_latest_docket_date_mask, :]

# Reshape from wide to long.
df = pd.melt(df,
             id_vars=['case_number', 'latest_docket_date', 'judgment_for_plaintiff'],
             value_vars=value_vars, var_name='month', value_name='zestimate')
df = df.sort_values(by=['case_number', 'month'])

# Get 'first treated' date for each observation.
df.loc[:, 'latest_docket_date'] = pd.to_datetime(df['latest_docket_date']).dt.strftime('%Y-%m')

# Convert months to integers.
df.loc[:, 'month'] = df['month'].replace(month_to_int_dictionary)
df.loc[:, 'latest_docket_date'] = df['latest_docket_date'].replace(month_to_int_dictionary)
never_treated_mask = (df['judgment_for_plaintiff'] == 0)
df.loc[never_treated_mask, 'latest_docket_date'] = np.NaN

# Set index to entity and time.
df = df.set_index(['case_number', 'month'])

Limiting sample to 2948 evictions for which we have Zestimate data.
Limiting sample to 2657 evictions for which the latest docket date is listed.


In [16]:
# Run DiD.
att_gt = ATTgt(data=df, cohort_name='latest_docket_date', freq='M')
result = att_gt.fit(formula='zestimate', n_jobs=-1)

Computing ATTgt [workers=10]  100%|████████████████████| 5040/5040 [01:14<00:00,


In [17]:
att_gt.aggregate('event')

EventAggregation                                     \
                                      analytic pointwise conf. band   
                             ATT     std_error                lower   
relative_period                                                       
-118                 6969.109705    452.771642          6081.693593   
-117                -1993.914487   3063.487794         -7998.240231   
-116                  639.717487   1575.974863         -2449.136485   
-115                 -267.788007   1547.716529         -3301.256661   
-114                 -273.006781   1246.594744         -2716.287582   
...                          ...           ...                  ...   
 38                  7069.112134   6062.269430         -4812.717612   
 39                  2973.271943   7819.897437        -12353.445396   
 40                  5228.935114  10753.984191        -15848.486590   
 41                 13828.340055  16214.266103        -17951.037542   
 42                 78253.953173  51670.519986        -23018.405063   

                                                  
                                                  
                         upper zero_not_in_cband  
relative_period                                   
-118               7856.525816                 *  
-117               4010.411257                    
-116               3728.571460                    
-115               2765.680648                    
-114               2170.274020                    
...                        ...               ...  
 38               18950.941880                    
 39               18299.989281                    
 40               26306.356818                    
 41               45607.717652                    
 42              179526.311408                    

[161 rows x 5 columns]

In [18]:
att_gt.plot('event')

alt.HConcatChart(...)